تست با bert و senticnet به همراه tada منعطف

In [1]:
!pip install transformers --upgrade
!pip isntall datasets --upgrade
!pip install senticnet
!pip install accelerate

ERROR: unknown command "isntall" - maybe you meant "install"
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 MB 4.1 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
# --- استفاده از AutoModel ---
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, AutoConfig
from torch.optim import AdamW
from datasets import load_dataset, Dataset as HFDataset # Rename to avoid conflict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from senticnet.senticnet import SenticNet
import re
import os
from tqdm.notebook import tqdm

In [3]:
# غیرفعال کردن هشدار توکنایزر
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("کتابخانه‌ها نصب و وارد شدند.")

کتابخانه‌ها نصب و وارد شدند.


In [4]:
class Config:
    # --- تغییر مدل به BERT ---
    MODEL_NAME = "bert-base-uncased" # Using BERT base
    # --- بقیه تنظیمات ---
    DATASET_NAME = "stanfordnlp/sst2"
    SUBSET_SIZE_TRAIN = 1000
    SUBSET_SIZE_VAL = 200
    RANDOM_STATE = 42
    TRAIN_BATCH_SIZE = 8 # BERT might fit with 8 or 16 on T4
    VALID_BATCH_SIZE = 8
    EPOCHS = 4
    LEARNING_RATE = 2e-5
    MAX_LEN = 128
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"در حال استفاده از دستگاه: {Config.DEVICE}")
print(f"مدل پایه: {Config.MODEL_NAME}") # Updated model name
print(f"دیتاست: {Config.DATASET_NAME} (Train: {Config.SUBSET_SIZE_TRAIN}, Val: {Config.SUBSET_SIZE_VAL})")
print(f"Batch Size: {Config.TRAIN_BATCH_SIZE}")
print("روش انطباق: TADA انعطاف‌پذیر (Flexible TADA - Proposed)")

در حال استفاده از دستگاه: cuda
مدل پایه: bert-base-uncased
دیتاست: stanfordnlp/sst2 (Train: 1000, Val: 200)
Batch Size: 8
روش انطباق: TADA انعطاف‌پذیر (Flexible TADA - Proposed)


In [5]:
class SymbolicModule:
    def __init__(self):
        try:
            self.sn = SenticNet()
        except Exception as e:
            print(f"خطا در بارگذاری SenticNet: {e}")
            self.sn = None

    def get_text_polarity(self, text):
        if self.sn is None: return 0.0, []
        text = re.sub(r'[^\w\s]', '', text)
        words = re.findall(r'\b\w+\b', text.lower())
        polarity_sum, word_count, keywords = 0, 0, []
        if not words: return 0.0, []
        for word in words:
            try:
                polarity_value_str = self.sn.polarity_value(word)
                polarity_sum += float(polarity_value_str)
                word_count += 1
                keywords.append(word)
            except KeyError: continue
            except Exception: continue
        if word_count == 0: return 0.0, []
        avg_polarity = np.clip(polarity_sum / word_count, -1.0, 1.0)
        return avg_polarity, list(set(keywords))

# تست اولیه ماژول
test_module_bert = SymbolicModule()
test_polarity_bert, test_keywords_bert = test_module_bert.get_text_polarity("This is a wonderfully good movie, not bad at all!")
# print(f"تست SenticNet - Polarity: {test_polarity_bert}, Keywords: {test_keywords_bert}") # Keep commented

In [6]:
def load_and_prepare_data(config, tokenizer):
    print(f"\nدر حال بارگذاری دیتاست {config.DATASET_NAME}...")
    try:
        dataset = load_dataset(config.DATASET_NAME)
    except Exception as e:
        print(f"خطا در بارگذاری دیتاست: {e}.")
        return None, None, None

    print("ساختار دیتاست:", dataset)
    if 'train' not in dataset or 'validation' not in dataset or \
       'sentence' not in dataset['train'].column_names or \
       'label' not in dataset['train'].column_names:
        print("خطا: ساختار دیتاست نامعتبر است.")
        return None, None, None

    try:
        train_df = dataset['train'].to_pandas().sample(n=config.SUBSET_SIZE_TRAIN, random_state=config.RANDOM_STATE)
        val_df = dataset['validation'].to_pandas().sample(n=config.SUBSET_SIZE_VAL, random_state=config.RANDOM_STATE)
        print(f"اندازه دیتاست آموزش: {len(train_df)}, اعتبارسنجی: {len(val_df)}")
    except ValueError as e:
         print(f"خطا در نمونه‌برداری: {e}.")
         if 'validation' in dataset and config.SUBSET_SIZE_VAL > len(dataset['validation']):
             print(f"استفاده از کل دیتاست اعتبارسنجی ({len(dataset['validation'])}) به جای نمونه‌برداری.")
             val_df = dataset['validation'].to_pandas()
         else: return None, None, None

    symbolic_module = SymbolicModule()

    class SentimentDatasetWithSentic(Dataset):
        def __init__(self, texts, labels, tokenizer, max_len, symbolic_module):
            self.texts, self.labels, self.tokenizer = texts, labels, tokenizer
            self.max_len, self.symbolic_module = max_len, symbolic_module

        def __len__(self): return len(self.texts)

        def __getitem__(self, idx):
            text, label = str(self.texts[idx]), int(self.labels[idx])
            symbolic_polarity, _ = self.symbolic_module.get_text_polarity(text)
            # BERT uses token_type_ids, although for single sentences they are not strictly necessary
            encoding = self.tokenizer.encode_plus(
                text, add_special_tokens=True, max_length=self.max_len, padding='max_length',
                truncation=True, return_token_type_ids=True, # Enable for BERT
                return_attention_mask=True, return_tensors='pt',
            )
            return {
                'input_ids': encoding['input_ids'].flatten().cpu(),
                'attention_mask': encoding['attention_mask'].flatten().cpu(),
                 # 'token_type_ids': encoding['token_type_ids'].flatten().cpu(), # Add if model needs it explicitly
                'labels': torch.tensor(label, dtype=torch.long).cpu(),
                'symbolic_features': torch.tensor([symbolic_polarity], dtype=torch.float).cpu()
            }

    train_dataset = SentimentDatasetWithSentic(train_df['sentence'].tolist(), train_df['label'].tolist(), tokenizer, config.MAX_LEN, symbolic_module)
    val_dataset = SentimentDatasetWithSentic(val_df['sentence'].tolist(), val_df['label'].tolist(), tokenizer, config.MAX_LEN, symbolic_module)
    train_loader = DataLoader(train_dataset, batch_size=config.TRAIN_BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=config.VALID_BATCH_SIZE, shuffle=False, num_workers=2)

    print("آماده‌سازی داده‌ها تکمیل شد.")
    return train_loader, val_loader, dataset

In [7]:
class NeuroSymbolicBertModelFlexibleTADA(torch.nn.Module): # اسم کلاس تغییر کرد
    def __init__(self, model_name, n_classes=2):
        super(NeuroSymbolicBertModelFlexibleTADA, self).__init__()
        print(f"\nدر حال بارگذاری مدل پایه {model_name} برای TADA انعطاف‌پذیر...")
        try:
            bert_config = AutoConfig.from_pretrained(model_name)
            self.bert = AutoModel.from_pretrained(model_name) # Use AutoModel
            print("مدل پایه BERT با موفقیت بارگذاری شد.")
        except Exception as e:
            print(f"خطا در بارگذاری مدل {model_name}: {e}")
            raise

        # --- پیاده‌سازی Flexible TADA ---
        print("در حال اعمال Flexible TADA (Embedding + آخرین لایه)...")
        # ۱. فریز کردن تمام پارامترها
        for param in self.bert.parameters():
            param.requires_grad = False

        # ۲. باز کردن لایه Embedding (نام استاندارد در BERT: 'embeddings')
        embedding_layer_name = 'embeddings'
        try:
            if hasattr(self.bert, embedding_layer_name):
                 embedding_layer = getattr(self.bert, embedding_layer_name)
            else:
                 raise AttributeError(f"لایه Embedding ('{embedding_layer_name}') در مدل BERT یافت نشد.")

            for param in embedding_layer.parameters():
                param.requires_grad = True
            print(f"پارامترهای '{embedding_layer_name}' برای آموزش باز شدند.")

        except AttributeError as e:
            print(f"خطا در یافتن لایه Embedding: {e}")
            raise

        # ۳. باز کردن آخرین لایه Transformer (نام استاندارد: 'encoder.layer')
        try:
            blocks_name = 'layer'
            parent_module = self.bert.encoder # Layers are under 'encoder' in BERT
            if hasattr(parent_module, blocks_name) and isinstance(getattr(parent_module, blocks_name), torch.nn.ModuleList):
                transformer_blocks = getattr(parent_module, blocks_name)
                print(f"بلوک‌های Transformer یافت شدند: encoder.{blocks_name}")
            else:
                 raise AttributeError(f"بلوک‌های Transformer ('encoder.{blocks_name}') در مدل BERT یافت نشد.")

            if not transformer_blocks or len(transformer_blocks) == 0:
                 raise ValueError("لیست بلوک‌های Transformer خالی است.")

            last_layer_index = len(transformer_blocks) - 1
            last_layer_name_full = f'encoder.{blocks_name}.{last_layer_index}'

            for param in transformer_blocks[last_layer_index].parameters():
                param.requires_grad = True
            print(f"پارامترهای آخرین لایه Transformer ('{last_layer_name_full}') برای آموزش باز شدند.")
            print("اعمال Flexible TADA تکمیل شد.")

        except (AttributeError, ValueError) as e:
             print(f"خطا در باز کردن آخرین لایه Transformer: {e}")
             raise

        # --- آداپتور و طبقه‌بند ---
        self.bert_hidden_size = bert_config.hidden_size # Usually 768 for bert-base
        self.classifier = torch.nn.Linear(self.bert_hidden_size + 1, n_classes)
        print(f"طبقه‌بند با ورودی {self.bert_hidden_size + 1} و خروجی {n_classes} ایجاد شد.")

    # متد forward (استفاده از خروجی [CLS] توکن برای BERT)
    def forward(self, input_ids, attention_mask, symbolic_features):
        # BERT typically uses token_type_ids, pass them if available in the input dict
        # token_type_ids = kwargs.get('token_type_ids', None) # Get if provided
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # token_type_ids=token_type_ids # Pass if needed
        )
        # --- BERT uses the output of the [CLS] token (first token) ---
        cls_output = bert_outputs.last_hidden_state[:, 0]
        # --- End BERT specific change ---

        combined_features = torch.cat([cls_output, symbolic_features.to(cls_output.device)], dim=1)
        logits = self.classifier(combined_features)
        return logits

In [8]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    total_loss, total_corrects = 0, 0
    progress_bar = tqdm(data_loader, desc="Epoch Train", leave=False, unit="batch")
    for d in progress_bar:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)
        symbolic_features = d["symbolic_features"].to(device)
        # Add token_type_ids if the model forward requires it
        # token_type_ids = d.get("token_type_ids", None)
        # if token_type_ids is not None: token_type_ids = token_type_ids.to(device)

        # Pass necessary inputs to the model
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        symbolic_features=symbolic_features)
                        # token_type_ids=token_type_ids) # Pass if needed

        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        _, preds = torch.max(outputs, dim=1)
        total_corrects += torch.sum(preds == labels)
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    avg_loss = total_loss / len(data_loader)
    accuracy = total_corrects.double() / len(data_loader.dataset)
    return accuracy, avg_loss

# تابع eval_model بدون تغییر...
def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    total_loss, total_corrects = 0, 0
    all_labels, all_preds = [], []
    progress_bar = tqdm(data_loader, desc="Validation", leave=False, unit="batch")
    with torch.no_grad():
        for d in progress_bar:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            symbolic_features = d["symbolic_features"].to(device)
            # token_type_ids = d.get("token_type_ids", None)
            # if token_type_ids is not None: token_type_ids = token_type_ids.to(device)

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            symbolic_features=symbolic_features)
                            # token_type_ids=token_type_ids) # Pass if needed
            loss = loss_fn(outputs, labels)
            _, preds = torch.max(outputs, dim=1)
            total_corrects += torch.sum(preds == labels)
            total_loss += loss.item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            progress_bar.set_postfix(loss=loss.item())
    accuracy = total_corrects.double() / len(data_loader.dataset)
    avg_loss = total_loss / len(data_loader)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    return accuracy, avg_loss, precision, recall, f1

In [9]:
def calculate_infidelity(model, tokenizer, text, label, symbolic_module, device, max_len):
    model.eval()
    original_polarity, keywords = symbolic_module.get_text_polarity(text)
    encoding = tokenizer.encode_plus(text, return_tensors='pt', max_length=max_len, padding='max_length', truncation=True, return_token_type_ids=True) # Ensure type ids for BERT if needed
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    # token_type_ids = encoding.get('token_type_ids', None) # Get if returned
    # if token_type_ids is not None: token_type_ids = token_type_ids.to(device)
    symbolic_features = torch.tensor([[original_polarity]], dtype=torch.float).to(device)

    with torch.no_grad():
        original_output = model(input_ids=input_ids,
                                attention_mask=attention_mask,
                                symbolic_features=symbolic_features)
                                # token_type_ids=token_type_ids)
        original_log_probs = torch.nn.functional.log_softmax(original_output, dim=1)
        original_prob = torch.exp(original_log_probs[0, label]).item()
    explanation = f"Keywords found: {keywords if keywords else 'None'}"
    perturbed_text = text
    if keywords:
        for keyword in sorted(keywords, key=len, reverse=True):
            perturbed_text = re.sub(r'\b' + re.escape(keyword) + r'\b', '', perturbed_text)
    perturbed_text = re.sub(r'\s+', ' ', perturbed_text).strip()
    if not perturbed_text: return explanation, 0.0
    perturbed_polarity, _ = symbolic_module.get_text_polarity(perturbed_text)
    perturbed_encoding = tokenizer.encode_plus(perturbed_text, return_tensors='pt', max_length=max_len, padding='max_length', truncation=True, return_token_type_ids=True)
    p_input_ids = perturbed_encoding['input_ids'].to(device)
    p_attention_mask = perturbed_encoding['attention_mask'].to(device)
    # p_token_type_ids = perturbed_encoding.get('token_type_ids', None)
    # if p_token_type_ids is not None: p_token_type_ids = p_token_type_ids.to(device)
    p_symbolic_features = torch.tensor([[perturbed_polarity]], dtype=torch.float).to(device)

    with torch.no_grad():
        perturbed_output = model(input_ids=p_input_ids,
                                 attention_mask=p_attention_mask,
                                 symbolic_features=p_symbolic_features)
                                 # token_type_ids=p_token_type_ids)
        perturbed_log_probs = torch.nn.functional.log_softmax(perturbed_output, dim=1)
        perturbed_prob = torch.exp(perturbed_log_probs[0, label]).item()
    infidelity = (original_prob - perturbed_prob) ** 2
    return explanation, infidelity

In [10]:
def main_bert_flexible_tada(): # Renamed main function
    config = Config()
    try:
        # Load BERT tokenizer
        tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
        # BERT usually doesn't need pad token adjustment like causal models
        print("توکنایزر BERT با موفقیت بارگذاری شد.")
    except Exception as e:
        print(f"خطا در بارگذاری توکنایزر {config.MODEL_NAME}: {e}")
        return

    train_loader, val_loader, dataset = load_and_prepare_data(config, tokenizer)
    if train_loader is None or dataset is None:
        print("اجرا به دلیل خطا در بارگذاری داده متوقف شد.")
        return

    try:
        # --- استفاده از کلاس مدل BERT ---
        model = NeuroSymbolicBertModelFlexibleTADA(config.MODEL_NAME, n_classes=2).to(config.DEVICE)
    except Exception as e:
         print(f"اجرا به دلیل خطا در ساخت مدل متوقف شد: {e}")
         import traceback
         traceback.print_exc()
         return


    print("\n--- پارامترهای قابل آموزش (تایید Flexible TADA برای BERT) ---")
    trainable_params_count, total_params_count = 0, 0
    for name, param in model.named_parameters():
        total_params_count += param.numel()
        if param.requires_grad:
            print(f"  - {name} ({param.numel()} پارامتر)")
            trainable_params_count += param.numel()
    print(f"تعداد کل پارامترها: {total_params_count:,}")
    print(f"تعداد پارامترهای قابل آموزش: {trainable_params_count:,} ({trainable_params_count/total_params_count*100:.2f}%)")
    print("---------------------------------------------------------")

    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config.LEARNING_RATE)
    total_steps = len(train_loader) * config.EPOCHS
    num_warmup_steps = int(total_steps * 0.1)
    print(f"Total steps: {total_steps}, Warmup steps: {num_warmup_steps}")
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)
    loss_fn = torch.nn.CrossEntropyLoss().to(config.DEVICE)

    print("\nشروع فرآیند آموزش و ارزیابی (BERT + Flexible TADA)...") # عنوان تغییر کرد
    best_val_f1 = -1
    final_val_acc, final_val_loss, final_val_prec, final_val_recall, final_val_f1 = 0,0,0,0,0

    for epoch in range(config.EPOCHS):
        print(f"\n--- Epoch {epoch + 1}/{config.EPOCHS} ---")
        train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, config.DEVICE, scheduler)
        print(f"Epoch {epoch + 1} Training   | Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")
        val_acc, val_loss, val_prec, val_recall, val_f1 = eval_model(model, val_loader, loss_fn, config.DEVICE)
        print(f"Epoch {epoch + 1} Validation | Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")
        if epoch == config.EPOCHS - 1:
            final_val_acc, final_val_loss, final_val_prec, final_val_recall, final_val_f1 = val_acc, val_loss, val_prec, val_recall, val_f1
        # if val_f1 > best_val_f1: best_val_f1 = val_f1

    print("\nآموزش تکمیل شد.")
    print("\n================== نتایج نهایی ارزیابی (BERT + Flexible TADA) ==================") # عنوان تغییر کرد
    print(f"دقت (Accuracy): {final_val_acc:.4f}")
    print(f"دقت مثبت (Precision): {final_val_prec:.4f}")
    print(f"بازیابی (Recall): {final_val_recall:.4f}")
    print(f"معیار اف-وان (F1-Score): {final_val_f1:.4f}")
    print("=======================================================================\n")

    print("=============== نمونه‌های توضیح‌پذیری (Infidelity - BERT + Flexible TADA) ===============") # عنوان تغییر کرد
    if dataset is None:
        print("خطا: متغیر dataset تعریف نشده است.")
        return

    val_texts_df = dataset['validation'].to_pandas().sample(n=5, random_state=config.RANDOM_STATE + 1)
    sample_texts = list(zip(val_texts_df['sentence'], val_texts_df['label']))
    total_infidelity, num_samples = 0, 0
    final_symbolic_module = SymbolicModule()
    for text, label in sample_texts:
        if not isinstance(text, str) or not text.strip(): continue
        try:
            explanation, infidelity = calculate_infidelity(model, tokenizer, text, label, final_symbolic_module, config.DEVICE, config.MAX_LEN)
            print(f"Text: '{text}' (Actual Label: {label})")
            print(f"  -> Explanation: {explanation}")
            print(f"  -> Infidelity: {infidelity:.6f}")
            print("-" * 20)
            total_infidelity += infidelity
            num_samples += 1
        except Exception as e:
            print(f"خطا در محاسبه Infidelity برای متن: '{text}' - {e}")

    if num_samples > 0: print(f"\nمیانگین Infidelity برای {num_samples} نمونه: {total_infidelity / num_samples:.6f}")
    else: print("\nهیچ نمونه‌ای برای محاسبه Infidelity پردازش نشد.")
    print("==============================================================================\n")


# --- این بلاک برای اجرای تابع اصلی است ---
if __name__ == '__main__':
    try:
        # --- فراخوانی تابع اصلی BERT ---
        main_bert_flexible_tada()
    except Exception as e:
        print(f"\n !!! یک خطای پیش‌بینی نشده در اجرای اصلی رخ داد: {e}")
        import traceback
        traceback.print_exc()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 3 Validation | Loss: 0.3719 | Acc: 0.8300 | Prec: 0.8440 | Rec: 0.8440 | F1: 0.8440

--- Epoch 4/4 ---


Epoch Train:   0%|          | 0/125 [00:00<?, ?batch/s]

Epoch 4 Training   | Loss: 0.3089 | Accuracy: 0.8850


Validation:   0%|          | 0/25 [00:00<?, ?batch/s]

Epoch 4 Validation | Loss: 0.3721 | Acc: 0.8350 | Prec: 0.8519 | Rec: 0.8440 | F1: 0.8479

آموزش تکمیل شد.

================== نتایج نهایی ارزیابی (BERT + Flexible TADA) ==================
دقت (Accuracy): 0.8350
دقت مثبت (Precision): 0.8519
بازیابی (Recall): 0.8440
معیار اف-وان (F1-Score): 0.8479

=============== نمونه‌های توضیح‌پذیری (Infidelity - BERT + Flexible TADA) ===============
Text: 'one from the heart . ' (Actual Label: 1)
  -> Explanation: Keywords found: ['heart']
  -> Infidelity: 0.117812
--------------------
Text: 'nervous breakdowns are not entertaining . ' (Actual Label: 0)
  -> Explanation: Keywords found: ['entertaining', 'nervous']
  -> Infidelity: 0.030166
--------------------
Text: 'what 's surprising about full frontal is that despite its overt self-awareness , parts of the movie still manage to break past the artifice and thoroughly engage you . ' (Actual Label: 1)
  -> Explanation: Keywords found: ['full', 'manage', 'engage', 'frontal', 'break', 'overt', 'artifi

تست با roberta و senticnet به همراه tada منعطف

In [11]:
class Config:
    # --- تغییر مدل به RoBERTa ---
    MODEL_NAME = "roberta-base" # Using RoBERTa base
    # --- بقیه تنظیمات ---
    DATASET_NAME = "stanfordnlp/sst2"
    SUBSET_SIZE_TRAIN = 1000
    SUBSET_SIZE_VAL = 200
    RANDOM_STATE = 42
    TRAIN_BATCH_SIZE = 8 # RoBERTa should fit with 8 on T4
    VALID_BATCH_SIZE = 8
    EPOCHS = 4
    LEARNING_RATE = 2e-5
    MAX_LEN = 128
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"در حال استفاده از دستگاه: {Config.DEVICE}")
print(f"مدل پایه: {Config.MODEL_NAME}") # Updated model name
print(f"دیتاست: {Config.DATASET_NAME} (Train: {Config.SUBSET_SIZE_TRAIN}, Val: {Config.SUBSET_SIZE_VAL})")
print(f"Batch Size: {Config.TRAIN_BATCH_SIZE}")
print("روش انطباق: TADA انعطاف‌پذیر (Flexible TADA - Proposed)")

در حال استفاده از دستگاه: cuda
مدل پایه: roberta-base
دیتاست: stanfordnlp/sst2 (Train: 1000, Val: 200)
Batch Size: 8
روش انطباق: TADA انعطاف‌پذیر (Flexible TADA - Proposed)


In [12]:
class SymbolicModule:
    def __init__(self):
        try:
            self.sn = SenticNet()
        except Exception as e:
            print(f"خطا در بارگذاری SenticNet: {e}")
            self.sn = None

    def get_text_polarity(self, text):
        if self.sn is None: return 0.0, []
        text = re.sub(r'[^\w\s]', '', text)
        words = re.findall(r'\b\w+\b', text.lower())
        polarity_sum, word_count, keywords = 0, 0, []
        if not words: return 0.0, []
        for word in words:
            try:
                polarity_value_str = self.sn.polarity_value(word)
                polarity_sum += float(polarity_value_str)
                word_count += 1
                keywords.append(word)
            except KeyError: continue
            except Exception: continue
        if word_count == 0: return 0.0, []
        avg_polarity = np.clip(polarity_sum / word_count, -1.0, 1.0)
        return avg_polarity, list(set(keywords))

# تست اولیه ماژول
test_module_roberta = SymbolicModule()
test_polarity_roberta, test_keywords_roberta = test_module_roberta.get_text_polarity("This is a wonderfully good movie, not bad at all!")
print(f"تست SenticNet - Polarity: {test_polarity_roberta}, Keywords: {test_keywords_roberta}") # Keep commented

تست SenticNet - Polarity: 0.09699999999999998, Keywords: ['wonderfully', 'good', 'bad']


In [13]:
def load_and_prepare_data(config, tokenizer):
    print(f"\nدر حال بارگذاری دیتاست {config.DATASET_NAME}...")
    try:
        dataset = load_dataset(config.DATASET_NAME)
    except Exception as e:
        print(f"خطا در بارگذاری دیتاست: {e}.")
        return None, None, None

    print("ساختار دیتاست:", dataset)
    if 'train' not in dataset or 'validation' not in dataset or \
       'sentence' not in dataset['train'].column_names or \
       'label' not in dataset['train'].column_names:
        print("خطا: ساختار دیتاست نامعتبر است.")
        return None, None, None

    try:
        train_df = dataset['train'].to_pandas().sample(n=config.SUBSET_SIZE_TRAIN, random_state=config.RANDOM_STATE)
        val_df = dataset['validation'].to_pandas().sample(n=config.SUBSET_SIZE_VAL, random_state=config.RANDOM_STATE)
        print(f"اندازه دیتاست آموزش: {len(train_df)}, اعتبارسنجی: {len(val_df)}")
    except ValueError as e:
         print(f"خطا در نمونه‌برداری: {e}.")
         if 'validation' in dataset and config.SUBSET_SIZE_VAL > len(dataset['validation']):
             print(f"استفاده از کل دیتاست اعتبارسنجی ({len(dataset['validation'])}) به جای نمونه‌برداری.")
             val_df = dataset['validation'].to_pandas()
         else: return None, None, None

    symbolic_module = SymbolicModule()

    class SentimentDatasetWithSentic(Dataset):
        def __init__(self, texts, labels, tokenizer, max_len, symbolic_module):
            self.texts, self.labels, self.tokenizer = texts, labels, tokenizer
            self.max_len, self.symbolic_module = max_len, symbolic_module

        def __len__(self): return len(self.texts)

        def __getitem__(self, idx):
            text, label = str(self.texts[idx]), int(self.labels[idx])
            symbolic_polarity, _ = self.symbolic_module.get_text_polarity(text)
            # --- حذف return_token_type_ids ---
            encoding = self.tokenizer.encode_plus(
                text, add_special_tokens=True, max_length=self.max_len, padding='max_length',
                truncation=True, return_token_type_ids=False, # Set to False for RoBERTa
                return_attention_mask=True, return_tensors='pt',
            )
            return {
                'input_ids': encoding['input_ids'].flatten().cpu(),
                'attention_mask': encoding['attention_mask'].flatten().cpu(),
                'labels': torch.tensor(label, dtype=torch.long).cpu(),
                'symbolic_features': torch.tensor([symbolic_polarity], dtype=torch.float).cpu()
            }

    train_dataset = SentimentDatasetWithSentic(train_df['sentence'].tolist(), train_df['label'].tolist(), tokenizer, config.MAX_LEN, symbolic_module)
    val_dataset = SentimentDatasetWithSentic(val_df['sentence'].tolist(), val_df['label'].tolist(), tokenizer, config.MAX_LEN, symbolic_module)
    train_loader = DataLoader(train_dataset, batch_size=config.TRAIN_BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=config.VALID_BATCH_SIZE, shuffle=False, num_workers=2)

    print("آماده‌سازی داده‌ها تکمیل شد.")
    return train_loader, val_loader, dataset

In [14]:
class NeuroSymbolicRobertaModelFlexibleTADA(torch.nn.Module): # اسم کلاس تغییر کرد
    def __init__(self, model_name, n_classes=2):
        super(NeuroSymbolicRobertaModelFlexibleTADA, self).__init__()
        print(f"\nدر حال بارگذاری مدل پایه {model_name} برای TADA انعطاف‌پذیر...")
        try:
            roberta_config = AutoConfig.from_pretrained(model_name)
            # --- استفاده از self.roberta ---
            self.roberta = AutoModel.from_pretrained(model_name)
            print("مدل پایه RoBERTa با موفقیت بارگذاری شد.")
        except Exception as e:
            print(f"خطا در بارگذاری مدل {model_name}: {e}")
            raise

        # --- پیاده‌سازی Flexible TADA ---
        print("در حال اعمال Flexible TADA (Embedding + آخرین لایه)...")
        # ۱. فریز کردن تمام پارامترها
        for param in self.roberta.parameters():
            param.requires_grad = False

        # ۲. باز کردن لایه Embedding (نام استاندارد در RoBERTa: 'embeddings')
        embedding_layer_name = 'embeddings'
        try:
            if hasattr(self.roberta, embedding_layer_name):
                 embedding_layer = getattr(self.roberta, embedding_layer_name)
            else:
                 raise AttributeError(f"لایه Embedding ('{embedding_layer_name}') در مدل RoBERTa یافت نشد.")

            for param in embedding_layer.parameters():
                param.requires_grad = True
            print(f"پارامترهای '{embedding_layer_name}' برای آموزش باز شدند.")

        except AttributeError as e:
            print(f"خطا در یافتن لایه Embedding: {e}")
            raise

        # ۳. باز کردن آخرین لایه Transformer (نام استاندارد: 'encoder.layer')
        try:
            blocks_name = 'layer'
            # --- استفاده از self.roberta.encoder ---
            parent_module = self.roberta.encoder
            if hasattr(parent_module, blocks_name) and isinstance(getattr(parent_module, blocks_name), torch.nn.ModuleList):
                transformer_blocks = getattr(parent_module, blocks_name)
                print(f"بلوک‌های Transformer یافت شدند: encoder.{blocks_name}")
            else:
                 raise AttributeError(f"بلوک‌های Transformer ('encoder.{blocks_name}') در مدل RoBERTa یافت نشد.")

            if not transformer_blocks or len(transformer_blocks) == 0:
                 raise ValueError("لیست بلوک‌های Transformer خالی است.")

            last_layer_index = len(transformer_blocks) - 1
            last_layer_name_full = f'encoder.{blocks_name}.{last_layer_index}'

            for param in transformer_blocks[last_layer_index].parameters():
                param.requires_grad = True
            print(f"پارامترهای آخرین لایه Transformer ('{last_layer_name_full}') برای آموزش باز شدند.")
            print("اعمال Flexible TADA تکمیل شد.")

        except (AttributeError, ValueError) as e:
             print(f"خطا در باز کردن آخرین لایه Transformer: {e}")
             raise

        # --- آداپتور و طبقه‌بند ---
        self.roberta_hidden_size = roberta_config.hidden_size # Usually 768 for roberta-base
        self.classifier = torch.nn.Linear(self.roberta_hidden_size + 1, n_classes)
        print(f"طبقه‌بند با ورودی {self.roberta_hidden_size + 1} و خروجی {n_classes} ایجاد شد.")

    # متد forward (استفاده از خروجی <s> توکن اول، شبیه [CLS] در BERT)
    def forward(self, input_ids, attention_mask, symbolic_features):
        # --- استفاده از self.roberta ---
        roberta_outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # RoBERTa doesn't use token_type_ids
        )
        # --- RoBERTa uses the output of the <s> token (first token) like BERT's [CLS] ---
        cls_output = roberta_outputs.last_hidden_state[:, 0]
        # --- End RoBERTa specific change ---

        combined_features = torch.cat([cls_output, symbolic_features.to(cls_output.device)], dim=1)
        logits = self.classifier(combined_features)
        return logits

In [15]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler):
    model.train()
    total_loss, total_corrects = 0, 0
    progress_bar = tqdm(data_loader, desc="Epoch Train", leave=False, unit="batch")
    for d in progress_bar:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)
        symbolic_features = d["symbolic_features"].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        symbolic_features=symbolic_features)

        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        _, preds = torch.max(outputs, dim=1)
        total_corrects += torch.sum(preds == labels)
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    avg_loss = total_loss / len(data_loader)
    accuracy = total_corrects.double() / len(data_loader.dataset)
    return accuracy, avg_loss

# تابع eval_model بدون تغییر...
def eval_model(model, data_loader, loss_fn, device):
    model.eval()
    total_loss, total_corrects = 0, 0
    all_labels, all_preds = [], []
    progress_bar = tqdm(data_loader, desc="Validation", leave=False, unit="batch")
    with torch.no_grad():
        for d in progress_bar:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            symbolic_features = d["symbolic_features"].to(device)

            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            symbolic_features=symbolic_features)
            loss = loss_fn(outputs, labels)
            _, preds = torch.max(outputs, dim=1)
            total_corrects += torch.sum(preds == labels)
            total_loss += loss.item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            progress_bar.set_postfix(loss=loss.item())
    accuracy = total_corrects.double() / len(data_loader.dataset)
    avg_loss = total_loss / len(data_loader)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    return accuracy, avg_loss, precision, recall, f1

In [16]:
def calculate_infidelity(model, tokenizer, text, label, symbolic_module, device, max_len):
    model.eval()
    original_polarity, keywords = symbolic_module.get_text_polarity(text)
    # --- حذف return_token_type_ids ---
    encoding = tokenizer.encode_plus(text, return_tensors='pt', max_length=max_len, padding='max_length', truncation=True, return_token_type_ids=False)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    symbolic_features = torch.tensor([[original_polarity]], dtype=torch.float).to(device)

    with torch.no_grad():
        original_output = model(input_ids=input_ids,
                                attention_mask=attention_mask,
                                symbolic_features=symbolic_features)
        original_log_probs = torch.nn.functional.log_softmax(original_output, dim=1)
        original_prob = torch.exp(original_log_probs[0, label]).item()
    explanation = f"Keywords found: {keywords if keywords else 'None'}"
    perturbed_text = text
    if keywords:
        for keyword in sorted(keywords, key=len, reverse=True):
            perturbed_text = re.sub(r'\b' + re.escape(keyword) + r'\b', '', perturbed_text)
    perturbed_text = re.sub(r'\s+', ' ', perturbed_text).strip()
    if not perturbed_text: return explanation, 0.0
    perturbed_polarity, _ = symbolic_module.get_text_polarity(perturbed_text)
    perturbed_encoding = tokenizer.encode_plus(perturbed_text, return_tensors='pt', max_length=max_len, padding='max_length', truncation=True, return_token_type_ids=False)
    p_input_ids = perturbed_encoding['input_ids'].to(device)
    p_attention_mask = perturbed_encoding['attention_mask'].to(device)
    p_symbolic_features = torch.tensor([[perturbed_polarity]], dtype=torch.float).to(device)

    with torch.no_grad():
        perturbed_output = model(input_ids=p_input_ids,
                                 attention_mask=p_attention_mask,
                                 symbolic_features=p_symbolic_features)
        perturbed_log_probs = torch.nn.functional.log_softmax(perturbed_output, dim=1)
        perturbed_prob = torch.exp(perturbed_log_probs[0, label]).item()
    infidelity = (original_prob - perturbed_prob) ** 2
    return explanation, infidelity

In [17]:
def main_roberta_flexible_tada(): # Renamed main function
    config = Config()
    try:
        # Load RoBERTa tokenizer
        tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)
        print("توکنایزر RoBERTa با موفقیت بارگذاری شد.")
    except Exception as e:
        print(f"خطا در بارگذاری توکنایزر {config.MODEL_NAME}: {e}")
        return

    train_loader, val_loader, dataset = load_and_prepare_data(config, tokenizer)
    if train_loader is None or dataset is None:
        print("اجرا به دلیل خطا در بارگذاری داده متوقف شد.")
        return

    try:
        # --- استفاده از کلاس مدل RoBERTa ---
        model = NeuroSymbolicRobertaModelFlexibleTADA(config.MODEL_NAME, n_classes=2).to(config.DEVICE)
    except Exception as e:
         print(f"اجرا به دلیل خطا در ساخت مدل متوقف شد: {e}")
         import traceback
         traceback.print_exc()
         return


    print("\n--- پارامترهای قابل آموزش (تایید Flexible TADA برای RoBERTa) ---")
    trainable_params_count, total_params_count = 0, 0
    for name, param in model.named_parameters():
        total_params_count += param.numel()
        if param.requires_grad:
            print(f"  - {name} ({param.numel()} پارامتر)")
            trainable_params_count += param.numel()
    print(f"تعداد کل پارامترها: {total_params_count:,}")
    print(f"تعداد پارامترهای قابل آموزش: {trainable_params_count:,} ({trainable_params_count/total_params_count*100:.2f}%)")
    print("---------------------------------------------------------")

    optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=config.LEARNING_RATE)
    total_steps = len(train_loader) * config.EPOCHS
    num_warmup_steps = int(total_steps * 0.1)
    print(f"Total steps: {total_steps}, Warmup steps: {num_warmup_steps}")
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)
    loss_fn = torch.nn.CrossEntropyLoss().to(config.DEVICE)

    print("\nشروع فرآیند آموزش و ارزیابی (RoBERTa + Flexible TADA)...") # عنوان تغییر کرد
    best_val_f1 = -1
    final_val_acc, final_val_loss, final_val_prec, final_val_recall, final_val_f1 = 0,0,0,0,0

    for epoch in range(config.EPOCHS):
        print(f"\n--- Epoch {epoch + 1}/{config.EPOCHS} ---")
        train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, config.DEVICE, scheduler)
        print(f"Epoch {epoch + 1} Training   | Loss: {train_loss:.4f} | Accuracy: {train_acc:.4f}")
        val_acc, val_loss, val_prec, val_recall, val_f1 = eval_model(model, val_loader, loss_fn, config.DEVICE)
        print(f"Epoch {epoch + 1} Validation | Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")
        if epoch == config.EPOCHS - 1:
            final_val_acc, final_val_loss, final_val_prec, final_val_recall, final_val_f1 = val_acc, val_loss, val_prec, val_recall, val_f1
        # if val_f1 > best_val_f1: best_val_f1 = val_f1

    print("\nآموزش تکمیل شد.")
    print("\n================== نتایج نهایی ارزیابی (RoBERTa + Flexible TADA) ==================") # عنوان تغییر کرد
    print(f"دقت (Accuracy): {final_val_acc:.4f}")
    print(f"دقت مثبت (Precision): {final_val_prec:.4f}")
    print(f"بازیابی (Recall): {final_val_recall:.4f}")
    print(f"معیار اف-وان (F1-Score): {final_val_f1:.4f}")
    print("=========================================================================\n")

    print("=============== نمونه‌های توضیح‌پذیری (Infidelity - RoBERTa + Flexible TADA) ===============") # عنوان تغییر کرد
    if dataset is None:
        print("خطا: متغیر dataset تعریف نشده است.")
        return

    val_texts_df = dataset['validation'].to_pandas().sample(n=5, random_state=config.RANDOM_STATE + 1)
    sample_texts = list(zip(val_texts_df['sentence'], val_texts_df['label']))
    total_infidelity, num_samples = 0, 0
    final_symbolic_module = SymbolicModule()
    for text, label in sample_texts:
        if not isinstance(text, str) or not text.strip(): continue
        try:
            explanation, infidelity = calculate_infidelity(model, tokenizer, text, label, final_symbolic_module, config.DEVICE, config.MAX_LEN)
            print(f"Text: '{text}' (Actual Label: {label})")
            print(f"  -> Explanation: {explanation}")
            print(f"  -> Infidelity: {infidelity:.6f}")
            print("-" * 20)
            total_infidelity += infidelity
            num_samples += 1
        except Exception as e:
            print(f"خطا در محاسبه Infidelity برای متن: '{text}' - {e}")

    if num_samples > 0: print(f"\nمیانگین Infidelity برای {num_samples} نمونه: {total_infidelity / num_samples:.6f}")
    else: print("\nهیچ نمونه‌ای برای محاسبه Infidelity پردازش نشد.")
    print("=================================================================================\n")


# --- این بلاک برای اجرای تابع اصلی است ---
if __name__ == '__main__':
    try:
        # --- فراخوانی تابع اصلی RoBERTa ---
        main_roberta_flexible_tada()
    except Exception as e:
        print(f"\n !!! یک خطای پیش‌بینی نشده در اجرای اصلی رخ داد: {e}")
        import traceback
        traceback.print_exc()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch 3 Validation | Loss: 0.6281 | Acc: 0.6250 | Prec: 0.5934 | Rec: 0.9908 | F1: 0.7423

--- Epoch 4/4 ---


Epoch Train:   0%|          | 0/125 [00:00<?, ?batch/s]

Epoch 4 Training   | Loss: 0.5985 | Accuracy: 0.7090


Validation:   0%|          | 0/25 [00:00<?, ?batch/s]

Epoch 4 Validation | Loss: 0.5994 | Acc: 0.7700 | Prec: 0.7266 | Rec: 0.9266 | F1: 0.8145

آموزش تکمیل شد.

================== نتایج نهایی ارزیابی (RoBERTa + Flexible TADA) ==================
دقت (Accuracy): 0.7700
دقت مثبت (Precision): 0.7266
بازیابی (Recall): 0.9266
معیار اف-وان (F1-Score): 0.8145

=============== نمونه‌های توضیح‌پذیری (Infidelity - RoBERTa + Flexible TADA) ===============
Text: 'one from the heart . ' (Actual Label: 1)
  -> Explanation: Keywords found: ['heart']
  -> Infidelity: 0.022040
--------------------
Text: 'nervous breakdowns are not entertaining . ' (Actual Label: 0)
  -> Explanation: Keywords found: ['entertaining', 'nervous']
  -> Infidelity: 0.000718
--------------------
Text: 'what 's surprising about full frontal is that despite its overt self-awareness , parts of the movie still manage to break past the artifice and thoroughly engage you . ' (Actual Label: 1)
  -> Explanation: Keywords found: ['full', 'manage', 'engage', 'frontal', 'break', 'overt', '